In [37]:
import numpy as np 
import tensorflow as tf 
  
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import LSTM 
  
from keras.optimizers import RMSprop 
  
from keras.callbacks import LambdaCallback 
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import ReduceLROnPlateau 
import random 
import sys

2024-04-09 16:15:31.786467: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 16:15:31.921110: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 16:15:31.922349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 16:15:32.704456: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:
# with open('data/article.txt', 'r') as file: 
#     text = file.read()     
# print(text) 

In [39]:
text = "Please remember that all group members must be present for the presentation. Should there be any issues with the schedule, please reach out to us."
# Storing all the unique characters present in the text 
vocabulary = sorted(list(set(text)))
print(len(vocabulary))
# vocabulary

25


In [40]:
# Creating dictionaries to map each character to an index 
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary)) 
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary)) 

In [41]:
max_length = 100
steps = 5
sentences = [] 
next_chars = []
for i in range(0, len(text) - max_length, steps): 
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
print(sentences)
print(next_chars)

['Please remember that all group members must be present for the presentation. Should there be any iss', 'e remember that all group members must be present for the presentation. Should there be any issues w', 'ember that all group members must be present for the presentation. Should there be any issues with t', ' that all group members must be present for the presentation. Should there be any issues with the sc', ' all group members must be present for the presentation. Should there be any issues with the schedul', 'group members must be present for the presentation. Should there be any issues with the schedule, pl', ' members must be present for the presentation. Should there be any issues with the schedule, please ', 'ers must be present for the presentation. Should there be any issues with the schedule, please reach', 'ust be present for the presentation. Should there be any issues with the schedule, please reach out ', 'e present for the presentation. Should there be any issues wit

In [42]:
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype=bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool)

In [43]:
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

## Building the LSTM ##

In [ ]:
model = Sequential() 
model.add(LSTM())